In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm,trange
import torch.nn as nn
from random import shuffle


In [4]:
df=pd.read_csv('100計數_地段率1209.csv',encoding='UTF-8')

In [6]:
data=np.zeros((141,132,50))
label=np.zeros((141,132,1))
for x in range(len(data)):
    for y in range(len(data[0])):
        id=141*y+x
        datadict={
            0:'距最近火車站距離',
            1:'活動中心',
            2:'郵局',
            3:'公車站',
            4:'加油站',
            5:'停車場',
            6:'腳踏車',
            7:'寺廟',
            8:'大賣場',
            9:'服飾',
            10:'便利商店',
            11:'超市美妝',
            12:'電子賣場',
            13:'銀行',
            14:'ATM',
            15:'公園',
            16:'高中',
            17:'國小',
            18:'國中',
            19:'補習班',
            20:'食物',
            21:'診所',
            22:'大學',
            23:'觀光景點',
            24:'禮品百貨',
            25:'醫院',
            26:'嫌惡設施_危險',
            27:'嫌惡設施_殯葬',
            28:'嫌惡設施_髒亂',
            29:'平均公告現值',
            30:'加權容積率',
            31:'加權建蔽率',
            32:'國土利用_農業',
            33:'國土利用_森林',
            34:'國土利用_交通',
            35:'國土利用_水利',
            36:'國土利用_商業',
            37:'國土利用_純住宅',
            38:'國土利用_混合住宅',
            39:'國土利用_製造業',
            40:'國土利用_倉儲',
            41:'國土利用_宗教',
            42:'國土利用_殯葬',
            43:'國土利用_其他建築用地',
            44:'國土利用_學校',
            45:'國土利用_醫療',
            46:'國土利用_其他公共利用土地',
            47:'國土利用_遊憩',
            48:'國土利用_礦業',
            49:'國土利用_其他',

            
        }
        label[x][y][0]=df.iloc[id]['內插地段率']
        for i in range(50):
            data[x][y][i]=df.iloc[id][datadict[i]]
            if  data[x][y][i]>0:
                temp=1
            else:
                data[x][y][i]=0


In [17]:


validnum=len(df[df['內插地段率']>0.1])
print(validnum)
validdata1=np.zeros((len(df[df['內插地段率']>0.1]),21,11,11))
validdata2=np.zeros((len(df[df['內插地段率']>0.1]),7,21,21))
validdata3=np.zeros((len(df[df['內插地段率']>0.1]),22))

validlabel=np.zeros((len(df[df['內插地段率']>0.1]),1))
counter=0
for y in trange(len(data[0])):
    for x in range(len(data)):
        if label[x][y][0]>0.5:
            for i in range(1,22):
                tempx=x-5
                tempy=y-5
                for a in range(11):
                    for b in range(11):
                        try:
                            validdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                        except:
                            continue
            for i in range(22,29):
                tempx=x-10
                tempy=y-10
                for a in range(21):
                    for b in range(21):
                        try:
                            validdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                        except:
                            continue
            
            validlabel[counter][0]=label[x][y][0]-100
            validdata3[counter][0]=data[x][y][0]
            for i in range(29,50):
                validdata3[counter][i-28]=data[x][y][i]
            counter+=1



            

9484


  0%|          | 0/132 [00:00<?, ?it/s]

In [8]:
from torch.utils.data import Dataset, DataLoader
class ExampleDataset(Dataset):

    # data loading
    def __init__(self,tr1,tr2,tr3,target, length):
        
        self.x1 = tr1.astype(np.float32)
        self.x2 = tr2.astype(np.float32)
        self.x3 = tr3.astype(np.float32)
        self.y = target.astype(np.float32)
        self.n_samples = length

    # working for indexing
    def __getitem__(self, index):
        
        return self.x1[index],self.x2[index],self.x3[index], self.y[index]

    # return the length of our dataset
    def __len__(self):
        
        return self.n_samples

In [18]:
k=[]

for i in range(len(validdata3)):
    k.append([validdata1[i],validdata2[i],validdata3[i],validlabel[i]])


r=np.random.permutation(k)
for i in range(len(validdata3)):
    validdata1[i]=k[i][0]
    validdata2[i]=k[i][1]
    validdata3[i]=k[i][2]
    validlabel[i]=k[i][3]
tr0_set=ExampleDataset(validdata1[0:8500],validdata2[0:8500],validdata3[0:8500],validlabel[0:8500],8500)
tr0_val=ExampleDataset(validdata1[8500:],validdata2[8500:],validdata3[8500:],validlabel[8500:],len(validdata3)-8500)
#
dataloader = DataLoader(tr0_set, batch_size=40, shuffle=True)
dataloader_val = DataLoader(tr0_val, batch_size=1, shuffle=False)

C:\Users\yehsanity\AppData\Local\Temp\ipykernel_15076\1992625236.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  r=np.random.permutation(k)


In [13]:
import torch.nn as nn
class NeuralNet(nn.Module):
    ''' A simple fully-connected deep neural network '''
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.cv2_1=nn.Conv2d(21, 64, 3, stride=1)
        self.cv2_2=nn.Conv2d(64, 256, 5, stride=1)
        self.cv2_3=nn.Conv2d(7, 32, 5, stride=1)
        self.cv2_4=nn.Conv2d(32, 64, 10, stride=1)

        self.linear1 = nn.Linear(22, 32)
        self.linear2 = nn.Linear(32, 64)

        self.layer1 = nn.Linear(5*5*256+8*8*64+64, 128)
        self.layer2 = nn.Linear(128, 32)
        self.out = nn.Linear(32, 1) 

        self.ba1=nn.BatchNorm3d(21)
        self.ba2=nn.BatchNorm3d(7)
        self.ba3=nn.BatchNorm1d(2)

        self.act_fn1 = nn.ReLU()
        self.act_fn2 = nn.ReLU()
        self.act_fn3 = nn.ReLU()
        self.act_fn4 = nn.ReLU()
        self.act_fn5 = nn.ReLU()
        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        # self.net = nn.Sequential(
        #     nn.Conv2d(28, 64, 3, stride=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 256, 5, stride=1),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(5*5*256, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 32),
        #     nn.ReLU(),
        #     nn.Linear(32, 1) 
        # )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')

    def forward(self, x1,x2,x3):
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        x1=self.cv2_1(x1)
        x1=self.act_fn1(x1)
        x1=self.cv2_2(x1)
        x1=nn.Flatten()(x1)
        x2=self.cv2_3(x2)
        x2=self.act_fn2(x2)
        x2=self.cv2_4(x2)
        x2=nn.Flatten()(x2)
        x3=self.linear1(x3)
        x3=self.act_fn3(x3)
        x3=self.linear2(x3)
        x = torch.cat((x1,x2,x3), dim=1)
        x = self.layer1(x)
        x = self.act_fn4(x)
        x = self.layer2(x)
        x = self.act_fn5(x)
        out=self.out(x)
        
        return out

    def cal_loss(self, pred, target):
        ''' Calculate loss '''
        # TODO: you may implement L1/L2 regularization here
        lamb = 0.0003
        if lamb :
          l2_reg = torch.tensor(0.).to('cuda')
          for param in self.parameters() :
            l2_reg += torch.norm(param)
          return self.criterion(pred, target) + l2_reg*lamb
        else : 
          return self.criterion(pred, target)

In [20]:
import torch.optim as optim

def train(tr_set, val_set, model, device):
    ''' DNN training '''
    criterion = nn.MSELoss()
    n_epochs = 70  # Maximum number of epochs
    # Setup optimizer
    optimizer = optim.Adam(model.parameters(), lr=1E-5,weight_decay=0.00002)   

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    best_loss = 100000.0
    while epoch < n_epochs:
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0
        model.train()                           # set model to training mode
        for x1,x2,x3, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x1,x2,x3, labels = x1.to(device),x2.to(device),x3.to(device), y.to(device)   # move data to device (cpu/cuda)
            outputs = model(x1,x2,x3)                     # forward pass (compute output)
            mse_loss = criterion(outputs, labels)  # compute loss
            _, train_pred = torch.max(outputs, 1)
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            train_loss += mse_loss.item()

        # After each epoch, test your model on the validation (development) set.
        if len(val_set) > 0:
            model.eval() # set the model to evaluation mode
            with torch.no_grad():
                for i, data in enumerate(val_set):
                    x1,x2,x3, labels = data
                    # print(data)
                    x1,x2,x3, labels = x1.to(device),x2.to(device),x3.to(device), labels.to(device)
                    outputs = model(x1,x2,x3)
                    batch_loss = criterion(outputs, labels) 
                    _, val_pred = torch.max(outputs, 1) 
                    an= torch.max(labels) 
                    val_loss += batch_loss.item()

                print('[{:03d}/{:03d}]  Loss: {:3.6f} | loss: {:3.6f}'.format(
                    epoch + 1, n_epochs, train_loss/len(tr_set),  val_loss/len(val_set)
                ))

                # if the model improves, save a checkpoint at this epoch
                if val_loss < best_loss:
                    best_loss = val_loss
                    torch.save(model, "model_cnn_muti1210.pth")
                    print('saving model with loss {:.3f}'.format(best_loss/len(val_set)))
        else:
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
                epoch + 1, n_epochs, train_loss/len(tr_set), train_loss/len(tr_set)
            ))
        epoch += 1
    # if not validating, save the last epoch
    if len(val_set) == 0:
        torch.save(model, 'model.pth')
        print('saving model at last epoch')

    return min_mse, loss_record

In [21]:

model = NeuralNet().to('cuda')  # Construct model and move to device
model_loss, model_loss_record = train(dataloader, dataloader_val, model, 'cuda')




[001/070]  Loss: 177.897999 | loss: 119.306724
[002/070]  Loss: 79.884084 | loss: 58.529214
saving model with loss 58.529
[003/070]  Loss: 68.196808 | loss: 51.832908
saving model with loss 51.833
[004/070]  Loss: 63.645501 | loss: 48.159798
saving model with loss 48.160
[005/070]  Loss: 60.499710 | loss: 49.342694
[006/070]  Loss: 57.535521 | loss: 47.371032
saving model with loss 47.371
[007/070]  Loss: 55.162647 | loss: 46.195956
saving model with loss 46.196
[008/070]  Loss: 53.055661 | loss: 44.886973
saving model with loss 44.887
[009/070]  Loss: 51.543760 | loss: 43.701222
saving model with loss 43.701
[010/070]  Loss: 50.101362 | loss: 43.802491
[011/070]  Loss: 49.136758 | loss: 43.193187
saving model with loss 43.193
[012/070]  Loss: 47.666691 | loss: 42.178416
saving model with loss 42.178
[013/070]  Loss: 46.912673 | loss: 42.985817
[014/070]  Loss: 46.646532 | loss: 42.694877
[015/070]  Loss: 45.405834 | loss: 43.097334
[016/070]  Loss: 44.922274 | loss: 43.471854
[017/070

In [ ]:

model=torch.load("model_cnn_muti1210.pth")
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x1,x2,x3, y in tt_set:                            # iterate through the dataloader
        x1,x2,x3 = x1.to(device),x2.to(device),x3.to(device)                       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x1,x2,x3)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

count=0
prd_err=[]
prd_set=ExampleDataset(validdata1,validdata2,validdata3,validlabel,len(validlabel))
tt_set = DataLoader(prd_set, batch_size=1, shuffle=False)
preds = test(tt_set, model, 'cuda')  # predict COVID-19 cases with your model
for i in range(len(preds)):
    if preds[i] - validlabel[i] > 10 or preds[i] - validlabel[i] < -10:
        print("實際值:",validlabel[i],"預測結果:",preds[i],"id:",i)
        count+=1
        prd_err.append(i)


print("預測錯誤數量:",count)
prd_err=pd.DataFrame(prd_err)
prd_err.to_csv("prd_err_cnn.csv",index=False)



AttributeError: Can't get attribute 'NeuralNet' on <module '__main__'>

In [ ]:
#real data(all)


realdata1=np.zeros((len(df),21,11,11))
realdata2=np.zeros((len(df),7,21,21))
realdata3=np.zeros((len(df),22))

counter=0
for y in trange(len(data[0])):
    for x in range(len(data)):
        for i in range(1,22):
            tempx=x-5
            tempy=y-5
            for a in range(11):
                for b in range(11):
                    try:
                        realdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                    except:
                        continue
        for i in range(24,29):
            tempx=x-10
            tempy=y-10
            for a in range(21):
                for b in range(21):
                    try:
                        realdata1[counter][i][a][b]+=data[tempx+a][tempy+b][i]
                    except:
                        continue

        
        
        realdata3[counter][0]=data[x][y][0]
        realdata3[counter][1]=data[x][y][29]
        for i in range(29,50):
            realdata3[counter][i-28]=data[x][y][i]
        
        counter=counter+1



KeyError: 0

In [ ]:

prd_set2=ExampleDataset(realdata1,realdata2,realdata3,realdata3,len(realdata3))
tt_set = DataLoader(prd_set2, batch_size=1, shuffle=False)
preds = test(tt_set, model, 'cuda')  # predict COVID-19 cases with your model

In [ ]:
import math
err=[]
pred=[]
for i in range(len(preds)):
    ##err.append(abs(preds[i][0]-reallabel[i][0]))
    pred.append(preds[i][0]+100)


In [ ]:
print(len(pred))

18612


In [ ]:

df['預測值']=pred
df.to_csv('100計數_地段率1209.csv')

In [ ]:
import decimal
import json
with open('1210\100計數_地段率1209.geojson', newline='',encoding="utf-8") as jsonfile:
    data = json.load(jsonfile)

for i in range(len(preds)):
    data['features'][i]['properties']['預測值']=pred[i]

print(data['features'][0]['properties'])
with open('100計數_地段率1209.geojson_pre.geojson', 'w',encoding="utf-8") as json_obj:
    json.dump(data, json_obj, ensure_ascii = False, indent=4)




with open('100計數_地段率1209.geojson_pre.geojson', newline='',encoding="utf-8") as jsonfile2:
    data2 = json.load(jsonfile2)
print(data2['features'][0]['properties'])

{'id': 1.0, 'left': 13373240.3256, 'top': 2635136.2672999995, 'right': 13373340.325599998, 'bottom': 2635036.2673, '活動中心': 0.0, '郵局': 0.0, '公車站': 0.0, '火車站': 0.0, '加油站': 0.0, '停車場': 0.0, '腳踏車': 0.0, '寺廟': 0.0, '大賣場': 0.0, '服飾': 0.0, '便利商店': 0.0, '超市美妝': 0.0, '電子賣場': 0.0, '銀行': 0.0, 'ATM': 0.0, '大學': 0.0, '高中': 0.0, '國小': 0.0, '國中': 0.0, '補習班': 0.0, '嫌惡設施_危險': 0.0, '嫌惡設施_殯葬': 0.0, '嫌惡設施_髒亂': 0.0, '食物': 0.0, '診所': 0.0, '醫院': 0.0, '禮品百貨': 0.0, '觀光景點': 0.0, '房屋稅數量': 0.0, '地段率總和': 0.0, '折舊年數總和': 0.0, '平均地段率': None, '平均折舊年數': None, 'EPSG3857_X': 13373290.325599998, 'EPSG3857_Y': 2635086.2672999995, '距最近火車站距離': 8583.1767164421, '公園': 0.0, '土地單價': None, '預測值': 108.66443061828613}
{'id': 1.0, 'left': 13373240.3256, 'top': 2635136.2672999995, 'right': 13373340.325599998, 'bottom': 2635036.2673, '活動中心': 0.0, '郵局': 0.0, '公車站': 0.0, '火車站': 0.0, '加油站': 0.0, '停車場': 0.0, '腳踏車': 0.0, '寺廟': 0.0, '大賣場': 0.0, '服飾': 0.0, '便利商店': 0.0, '超市美妝': 0.0, '電子賣場': 0.0, '銀行': 0.0, 'ATM': 0.0, '大學': 0.0, '高中': 0.0, '國